# Spatial Permutation

Developed by Angelika Zarkali https://github.com/AngelikaZa

Aim: Run spatial permutation on sets of gene expression data with precomputed sphere permutations from R.

Atlas: Desikan-Cerebellum

In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import percentileofscore
import statsmodels.api as sm
#import statsmodels.formula.api as sm
from statsmodels.formula.api import ols
from pathlib import Path
import seaborn as sns
#import brainconn as con
import bct as bct
import networkx as nx
from sklearn.covariance import EmpiricalCovariance
from nilearn.connectome import ConnectivityMeasure
import nibabel
import nilearn
from nilearn import datasets, plotting, input_data, signal, image # datasets: for fetching atlas
from nilearn.input_data import NiftiLabelsMasker
# Enable inline plotting
%matplotlib inline

C:\Users\Angelika\Anaconda3\envs\Graph\lib\site-packages\nilearn\datasets\__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [67]:
#Permutation testing
## Precomputed permutations
permutation = pd.read_csv(r"permutations_DesikanCerebellum.txt", header = None, delimiter=" ")
## Data to run
df = pd.read_csv(r"data_angeliki_new_genes.csv")
df = df[["PC1", "HTT", "CYP26A1", "TM6SF1", "OSTN", "TSHZ3", "SSX2IP", "FBXW7", "SATB2", "PART1", "MCHR2", "RORB"]]
df.head(1)

,PC1,HTT,CYP26A1,TM6SF1,OSTN,TSHZ3,SSX2IP,FBXW7,SATB2,PART1,MCHR2,RORB
0,1.406176,0.635302,0.676595,0.629083,0.651484,0.81675,0.690345,0.824621,0.769496,0.840362,0.761164,0.744081


In [68]:
## First need to re-index the data to all Left regions then all Right regions
index_L = pd.read_csv("ids_DesikanCerebellum_L.txt", header=None)
index_L = index_L.values
index_R = pd.read_csv("ids_DesikanCerebellum_R.txt")
index_R = index_R.values
all_indices = np.append(index_L, index_R)
## arrange dataframe accordingly
df = df.reindex(all_indices)
df.head(5)

,PC1,HTT,CYP26A1,TM6SF1,OSTN,TSHZ3,SSX2IP,FBXW7,SATB2,PART1,MCHR2,RORB
42,0.964821,0.601363,0.643346,0.566846,0.607162,0.702210,0.689136,0.778267,0.781065,0.731163,0.644559,0.778510
43,1.673063,0.621177,0.632011,0.564071,0.529675,0.756539,0.709191,0.791779,0.815930,0.787926,0.648183,0.812391
44,1.239650,0.628651,0.666430,0.616078,0.642249,0.759448,0.714753,0.808496,0.814348,0.800255,0.715032,0.770681
45,0.920402,0.622527,0.677873,0.609475,0.612056,0.764546,0.745273,0.807120,0.822917,0.778080,0.677189,0.769962
46,0.822377,0.662069,0.630775,0.545792,0.646038,0.788955,0.750193,0.812910,0.826598,0.758129,0.689026,0.805114


In [70]:
# Run correlation for HTT

n_roi = 110 # nodes
n_perm = 1000 # permutations 
## data to correlate
df = df.replace(np.nan,0)
cor = df.PC1
gene = df.HTT

# Empty strings to hold permutations and results
r_spin = np.empty(n_perm)
r_obs, p_obs = stats.spearmanr(cor, gene)
r_spin = np.empty(n_perm)

for i in range(n_perm):
    rotated_index = permutation[i]
    perm = np.empty(n_roi)
    for j in range(n_roi):
        perm[j] = cor[(rotated_index[j])] # -1 as starting from 0
    r_spin[i] = stats.spearmanr(perm, gene)[0]
pv_spin = np.mean(np.abs(r_spin) >= np.abs(r_obs))
r_obs, p_obs, pv_spin

(0.5412136598740074, 1.031212627845307e-09, 0.0)

In [71]:
# Run correlation for each gene
genes = ["HTT","CYP26A1", "TM6SF1", "OSTN", "TSHZ3", "SSX2IP", "FBXW7", "SATB2", "PART1", "MCHR2", "RORB"]
n_roi = 110 # nodes
n_perm = 1000 # permutations 
## data to correlate
cor = df.PC1
geneData = pd.DataFrame(data=genes)
geneData["r"] = 0
geneData["p-obs"] = 0
geneData["p-spin"] = 0
for n in range(len(genes)): 
    gene = df[genes[n]]
    # Empty strings to hold permutations and results
    r_spin = np.empty(n_perm)
    r_obs, p_obs = stats.spearmanr(cor, gene)
    r_spin = np.empty(n_perm)

    for i in range(n_perm):
        rotated_index = permutation[i]
        perm = np.empty(n_roi)
        for j in range(n_roi):
            perm[j] = cor[(rotated_index[j])] # -1 as starting from 0
        r_spin[i] = stats.spearmanr(perm, gene)[0]
    pv_spin = np.mean(np.abs(r_spin) >= np.abs(r_obs))
    geneData["r"].iloc[n] = r_obs
    geneData["p-obs"].iloc[n] = p_obs
    geneData["p-spin"].iloc[n] = pv_spin
    
geneData.to_csv("Gene_Results.csv")

C:\Users\Angelika\Anaconda3\envs\Graph\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
